In [153]:
with open("../data/the_verdict.txt","r") as f:
    raw_text=f.read()

In [154]:
raw_text

'I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no great surprise to me to hear that, in the height of his glory, he had dropped his painting, married a rich widow, and established himself in a villa on the Riviera. (Though I rather thought it would have been Rome or Florence.)\n\n"The height of his glory"--that was what the women called it. I can hear Mrs. Gideon Thwing--his last Chicago sitter--deploring his unaccountable abdication. "Of course it\'s going to send the value of my picture \'way up; but I don\'t think of that, Mr. Rickham--the loss to Arrt is all I think of." The word, on Mrs. Thwing\'s lips, multiplied its _rs_ as though they were reflected in an endless vista of mirrors. And it was not only the Mrs. Thwings who mourned. Had not the exquisite Hermia Croft, at the last Grafton Gallery show, stopped me before Gisburn\'s "Moon-dancers" to say, with tears in her eyes: "We shall not look upon its like again"?\n\nWell!--even 

In [143]:
utf_encoding=list(raw_text.encode())

In [144]:
len(utf_encoding)

20479

In [5]:
def get_stats(ids):
    counts={}
    for pair in zip(ids,ids[1:]):
        counts[pair]=counts.get(pair,0)+1
    return counts

In [6]:
example="Hello, Guys Building LLM from Sratch"
example_encode=list(example.encode())

In [7]:
example_counts=get_stats(example_encode)
print(example_counts)
print(len(example_counts))

{(72, 101): 1, (101, 108): 1, (108, 108): 1, (108, 111): 1, (111, 44): 1, (44, 32): 1, (32, 71): 1, (71, 117): 1, (117, 121): 1, (121, 115): 1, (115, 32): 1, (32, 66): 1, (66, 117): 1, (117, 105): 1, (105, 108): 1, (108, 100): 1, (100, 105): 1, (105, 110): 1, (110, 103): 1, (103, 32): 1, (32, 76): 1, (76, 76): 1, (76, 77): 1, (77, 32): 1, (32, 102): 1, (102, 114): 1, (114, 111): 1, (111, 109): 1, (109, 32): 1, (32, 83): 1, (83, 114): 1, (114, 97): 1, (97, 116): 1, (116, 99): 1, (99, 104): 1}
35


In [8]:
max(example_counts,key=example_counts.get)

(72, 101)

In [9]:
def merge(ids, pair, idx):
  newids = []
  i = 0
  while i < len(ids):
    if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
      newids.append(idx)
      i += 2
    else:
      newids.append(ids[i])
      i += 1
  return newids

In [10]:
num_merges = 1000
vocabulary_size=256
merges = {} # (int, int) -> int
ids=list(utf_encoding)
for i in range(num_merges):
  stats = get_stats(ids)
  try:
    pair = max(stats, key=stats.get)
  except Exception as e:
    break
  idx = 256 + i
  print(f"merging {pair} into a new token {idx}")
  ids = merge(ids, pair, idx)
  merges[pair] = idx

merging (101, 32) into a new token 256
merging (32, 116) into a new token 257
merging (100, 32) into a new token 258
merging (116, 32) into a new token 259
merging (105, 110) into a new token 260
merging (115, 32) into a new token 261
merging (104, 256) into a new token 262
merging (104, 97) into a new token 263
merging (44, 32) into a new token 264
merging (111, 117) into a new token 265
merging (101, 114) into a new token 266
merging (97, 110) into a new token 267
merging (111, 110) into a new token 268
merging (101, 110) into a new token 269
merging (257, 262) into a new token 270
merging (121, 32) into a new token 271
merging (46, 32) into a new token 272
merging (111, 32) into a new token 273
merging (260, 103) into a new token 274
merging (104, 105) into a new token 275
merging (105, 116) into a new token 276
merging (101, 258) into a new token 277
merging (73, 32) into a new token 278
merging (111, 102) into a new token 279
merging (119, 97) into a new token 280
merging (115, 11

In [11]:
print("tokens length:", len(utf_encoding))
print("ids length:", len(ids))
print(f"compression ratio: {len(utf_encoding) / len(ids):.2f}X")

tokens length: 20479
ids length: 6156
compression ratio: 3.33X


In [12]:
len(merges)

1000

In [13]:
utf_to_token={}
token_to_utf={}
i=0
for utf in ids:
    if utf_to_token.get(utf) == None:
        utf_to_token[utf]=i
        token_to_utf[i]=utf
        i+=1

In [14]:
test="Build LLM From Scratch"
test_ids=list(test.encode())

In [15]:
print(test_ids)

[66, 117, 105, 108, 100, 32, 76, 76, 77, 32, 70, 114, 111, 109, 32, 83, 99, 114, 97, 116, 99, 104]


In [16]:
new_id=[]
i=0
while i < len(test_ids)-1:
    pair=(test_ids[i],test_ids[i+1])
    if merges.get(pair)!=None:
        new_id.append(utf_to_token[merges[pair]])
        i+=2
    else:
        new_id.append(utf_to_token[test_ids[i]])
        i+=1

In [17]:
new_id

[280, 110, 600, 308, 958, 958, 182, 16, 65, 638, 136, 16, 598, 866, 479, 799]

In [18]:
base_decoded_test_ids=[]
for id in new_id:
    base_decoded_test_ids.append(token_to_utf[id])

In [19]:
base_decoded_test_ids

[66, 117, 554, 258, 76, 76, 77, 32, 70, 333, 109, 32, 83, 887, 287, 325]

In [20]:
token_merge={}
for k,v in merges.items():
    token_merge[v]=k

In [21]:
def merged_token(pair, decode_ids=None):
    if decode_ids is None:
        decode_ids = []

    first, second = pair

    if first > 255:
        merged_token(token_merge[first], decode_ids)
    else:
        decode_ids.append(first)

    if second > 255:
        merged_token(token_merge[second], decode_ids)
    else:
        decode_ids.append(second)

    return decode_ids

In [22]:
decoded_test_ids = []

for id in new_id:
    tid = token_to_utf[id]
    if tid > 255:
        pair = token_merge[tid]
        base_utf = merged_token(pair)
        decoded_test_ids.extend(base_utf)
    else:
        decoded_test_ids.append(tid)


In [23]:
print(f"Text UTF encoding               {test_ids}")
print(f"Encoded ID from Tokenizer       {new_id}")
print(f"Decoder ID without BPE Decoding {base_decoded_test_ids}")
print(f"Final UTF encoding              {decoded_test_ids}")

Text UTF encoding               [66, 117, 105, 108, 100, 32, 76, 76, 77, 32, 70, 114, 111, 109, 32, 83, 99, 114, 97, 116, 99, 104]
Encoded ID from Tokenizer       [280, 110, 600, 308, 958, 958, 182, 16, 65, 638, 136, 16, 598, 866, 479, 799]
Decoder ID without BPE Decoding [66, 117, 554, 258, 76, 76, 77, 32, 70, 333, 109, 32, 83, 887, 287, 325]
Final UTF encoding              [66, 117, 105, 108, 100, 32, 76, 76, 77, 32, 70, 114, 111, 109, 32, 83, 99, 114, 97, 116, 99, 104]


In [24]:
bytes(decoded_test_ids).decode()

'Build LLM From Scratch'

In [ ]:
class TokenizerWithBPE:
    def __init__(self, merges):
        # Initialize dictionaries
        self.utf_to_token = {}
        self.token_to_utf = {}
        self.utf_to_merge = {}  # merged_token_id -> (first, second)
        self.merge_to_utf = {}  # (first, second) -> merged_token_id
        
        # Initialize base tokens (0-255)
        for utf in range(256):
            self.utf_to_token[utf] = utf
            self.token_to_utf[utf] = utf
        
        # Add merged tokens
        for pair, token_id in merges.items():
            self.merge_to_utf[pair] = token_id
            self.utf_to_merge[token_id] = pair
            self.utf_to_token[token_id] = token_id
            self.token_to_utf[token_id] = token_id
    
    def encode(self, text: str):
        # Convert text to UTF-8 bytes
        text_utf = list(text.encode('utf-8'))
        i = 0
        encode_id = []
        
        while i < len(text_utf):
            # If we're at the last byte, just add it
            if i == len(text_utf) - 1:
                encode_id.append(self.utf_to_token[text_utf[i]])
                break
            
            # Check if current pair exists in merges
            pair = (text_utf[i], text_utf[i+1])
            if pair in self.merge_to_utf:
                # Get the merged token ID
                token_id = self.merge_to_utf[pair]
                encode_id.append(self.utf_to_token[token_id])
                i += 2
            else:
                # No merge possible, add single byte
                encode_id.append(self.utf_to_token[text_utf[i]])
                i += 1
            
        return encode_id
    
    def _decode_token(self, token_id):
        """Recursively decode a token into base bytes"""
        result = []
        
        if token_id <= 255:
            # Base byte
            return [token_id]
        elif token_id in self.utf_to_merge:
            # Merged token - decode recursively
            first, second = self.utf_to_merge[token_id]
            result.extend(self._decode_token(first))
            result.extend(self._decode_token(second))
            return result
        else:
            raise ValueError(f"Unknown token ID: {token_id}")
    
    def decode(self, ids):
        decoded_bytes = []
        
        for token_id in ids:
            if token_id <= 255:
                decoded_bytes.append(token_id)
            else:
                # Recursively decode merged token
                base_bytes = self._decode_token(token_id)
                decoded_bytes.extend(base_bytes)
        
        # Convert bytes to string
        return bytes(decoded_bytes).decode('utf-8')

In [145]:
num_merges = 1000
vocabulary_size=256
merges = {} # (int, int) -> int
ids=list(utf_encoding)
for i in range(num_merges):
  stats = get_stats(ids)
  try:
    pair = max(stats, key=stats.get)
  except Exception as e:
    break
  idx = 256 + i
  ids = merge(ids, pair, idx)
  merges[pair] = idx

In [147]:
len(merges)

1000

In [148]:
tokenizer=TokenizerWithBPE(merges)

In [152]:
print(list("Build LLM From Scratch".encode()))

[66, 117, 105, 108, 100, 32, 76, 76, 77, 32, 70, 114, 111, 109, 32, 83, 99, 114, 97, 116, 99, 104]


In [149]:
token_id=tokenizer.encode("Build LLM From Scratch")

[66, 117, 554, 258, 76, 76, 77, 32, 70, 333, 109, 32, 83, 887, 287, 325]


In [150]:
tokenizer.decode(token_id)

'Build LLM From Scratch'

In [58]:
class TokenizerWithBPE:
    def __init__(self, vocab, merges):
        self.utf_to_token = {}
        self.token_to_utf = {}
        self.token_to_merge = {}  # Renamed for clarity
        self.merge_to_token = {}  # Added: token ID -> (pair1, pair2)
        token_length = 0

        # Initialize basic tokens (0-255)
        for utf in range(256):
            self.utf_to_token[utf] = utf
            self.token_to_utf[utf] = utf
        
        # Add merged tokens
        for (first, second), token_id in merges.items():
            self.utf_to_token[token_id] = token_id
            self.token_to_utf[token_id] = token_id
            self.token_to_merge[token_id] = (first, second)
            self.merge_to_token[(first, second)] = token_id

    def encode(self, text: str):
        text_utf = list(text.encode('utf-8'))
        i = 0
        encode_id = []
        
        while i < len(text_utf):
            # Try to find longest possible merge
            found = False
            # Check from longest possible pair to single token
            for length in range(min(2, len(text_utf) - i), 0, -1):
                if length == 2:
                    pair = (text_utf[i], text_utf[i+1])
                    if pair in self.merge_to_token:
                        encode_id.append(self.merge_to_token[pair])
                        i += 2
                        found = True
                        break
                else:  # length == 1
                    encode_id.append(self.utf_to_token[text_utf[i]])
                    i += 1
                    found = True
                    break
            
            if not found:
                encode_id.append(self.utf_to_token[text_utf[i]])
                i += 1
        
        return encode_id
    
    def _decode_token(self, token_id, decoded_ids=None):
        """Recursively decode a merged token into base bytes"""
        if decoded_ids is None:
            decoded_ids = []
        
        if token_id <= 255:
            # Base byte
            decoded_ids.append(token_id)
        elif token_id in self.token_to_merge:
            # Merged token - recursively decode
            first, second = self.token_to_merge[token_id]
            self._decode_token(first, decoded_ids)
            self._decode_token(second, decoded_ids)
        else:
            # Should not happen
            raise ValueError(f"Unknown token ID: {token_id}")
        
        return decoded_ids
    
    def decode(self, ids):
        decoded_bytes = []
        for token_id in ids:
            if token_id <= 255:
                # Direct byte
                decoded_bytes.append(token_id)
            else:
                # Merged token - decode recursively
                base_bytes = self._decode_token(token_id, [])
                decoded_bytes.extend(base_bytes)
        
        # Convert to bytes and decode to string
        return bytes(decoded_bytes).decode('utf-8')


# Helper functions (assuming these are defined elsewhere)
def get_stats(ids):
    """Count frequency of consecutive token pairs"""
    counts = {}
    for i in range(len(ids)-1):
        pair = (ids[i], ids[i+1])
        counts[pair] = counts.get(pair, 0) + 1
    return counts

def merge(ids, pair, idx):
    """Merge all occurrences of pair with new token idx"""
    new_ids = []
    i = 0
    while i < len(ids):
        if i < len(ids)-1 and (ids[i], ids[i+1]) == pair:
            new_ids.append(idx)
            i += 2
        else:
            new_ids.append(ids[i])
            i += 1
    return new_ids


# Example usage
if __name__ == "__main__":
    # Start with basic UTF-8 bytes of a sample text
    sample_text = "Hello World"
    utf_encoding = list(sample_text.encode('utf-8'))
    
    num_merges = 100
    merges = {}
    ids = utf_encoding.copy()
    
    # BPE training
    for i in range(num_merges):
        stats = get_stats(ids)
        if not stats:
            break
        
        # Find most frequent pair
        pair = max(stats, key=stats.get)
        idx = 256 + i  # New token ID
        
        print(f"Merging {pair} into new token {idx}")
        ids = merge(ids, pair, idx)
        merges[pair] = idx
    
    # Create tokenizer
    tokenizer = TokenizerWithBPE(range(256 + len(merges)), merges)
    
    # Test
    test_text = "Hello,GuysTestingBPETokenize"
    token_ids = tokenizer.encode(test_text)
    print(f"Token IDs: {token_ids}")
    
    decoded_text = tokenizer.decode(token_ids)
    print(f"Decoded: {decoded_text}")
    print(f"Original and decoded match: {test_text == decoded_text}")

Merging (72, 101) into new token 256
Merging (256, 108) into new token 257
Merging (257, 108) into new token 258
Merging (258, 111) into new token 259
Merging (259, 32) into new token 260
Merging (260, 87) into new token 261
Merging (261, 111) into new token 262
Merging (262, 114) into new token 263
Merging (263, 108) into new token 264
Merging (264, 100) into new token 265
Token IDs: [256, 108, 108, 111, 44, 71, 117, 121, 115, 84, 101, 115, 116, 105, 110, 103, 66, 80, 69, 84, 111, 107, 101, 110, 105, 122, 101]
Decoded: Hello,GuysTestingBPETokenize
Original and decoded match: True
